In [1]:
import numpy as np
import collections as col
from wingstructure import structure

# Define materials

In [2]:
Material = col.namedtuple('Material', ['ρ'])

fabric = Material(ρ=0.3e3)
foam = Material(ρ=0.18e3)

# Define structure

## import dat file

In [3]:
# load airfoil.dat file
coords = np.loadtxt('airfoils/ah93157.dat', skiprows=1)*1.2

# initialize base for section analysis
secbase = structure.SectionBase(coords)
secbase

## add structure elements

In [4]:
out = structure.Layer(fabric, 2e-3)
core = structure.Layer(fabric, 5e-3)
inner = structure.Layer(fabric, 2e-3)
secbase.append(out)
secbase.append(core)
secbase.append(inner)
secbase

In [5]:
spar = structure.ISpar({'flange': fabric, 'web': fabric}, 0.5, 0.3, 3e-2, 0.8, 5e-3)
secbase.append(spar)
secbase

In [6]:
%matplotlib ipympl
from IPython.display import Markdown
from ipywidgets import interact, interactive, ToggleButton, FloatSlider, IntSlider
import ipywidgets as widgets

In [15]:
w = widgets.Output()

ma = structure.MassAnalysis(secbase)

display(w)

class secshow:
    def __init__(self, secbase, ma):
        self.w = widgets.Output()
        self.secbase = secbase
        self.ma = ma
        
    def showding(self,t, t2, t3, p, p2):
        self.secbase.features[1].thickness=t2*1e-3
        self.secbase.features[3].flangethickness = t*1e-3
        self.secbase.features[3].webthickness = t3*1e-3
        self.secbase.features[3].midpos = p
        self.secbase.features[3].webpos = p2
        massprops = self.ma.massproperties
        with w:
            w.clear_output(wait=True)
            display(self.secbase)
            display('Masse/Länge: {:.2f} kg/m'.format(massprops[1]))

showding = secshow(secbase, ma)

shellthickness_slider = FloatSlider(value=1, min=0.5, max=25, step=2.5, description='Balsadicke')
flangethickness_slider = IntSlider(value=20, min=10, max=35, description='Holmdicke')
webthickness_slider = IntSlider(value=3, min=1, max=15, description='Stegbreite')
sparpos_slider = FloatSlider(value=0.5, min=0.3, max=0.7, step=0.05, description='Holmposition')
webpos_slider = FloatSlider(value=0.5, min=0.0, max=1.0, step=0.05, description='Stegposition')

interact(showding.showding, t=flangethickness_slider, t2=shellthickness_slider, 
         t3=webthickness_slider, p=sparpos_slider, p2=webpos_slider);

Output()

interactive(children=(IntSlider(value=20, description='Holmdicke', max=35, min=10), FloatSlider(value=1.0, des…

# Analyse mass

In [32]:
with open('test.svg','w') as afile:
    afile.write(secbase._repr_svg_())

In [64]:
import re
bounds_str = re.match(r'viewBoox', secbase._repr_svg_())
bounds_str

In [67]:
search_res = re.search(r'viewBox="((?:-?\d+.\d+\s*){4})"', secbase._repr_svg_())

In [69]:
res = search_res.groups()[0]

In [74]:
bounds = [float(val) for val in search_res.groups()[0].split()]
Δy = bounds[3]-bounds[1]
Δx = bounds[2]-bounds[0]
bounds

[-0.0478752, -0.0974352, 1.2958703999999999, 0.2842104]

In [60]:
secbase._repr_svg_();

In [76]:
def rework_svg(svg, width, height=100, stroke_width=None):
    import re
    
    if stroke_width is None:
        
        search_res = re.search(r'viewBox="((?:-?\d+.\d+\s*){4})"', svg)
        
        if search_res is None:
            raise Exception('Invalid SVG!')
            
        bounds = [float(val) for val in search_res.groups()[0].split()]
        Δy = bounds[3]-bounds[1]
        Δx = bounds[2]-bounds[0]
        
        stroke_width = min(Δx,Δy)/100
    
    svg = re.sub(r'width="\d+\.\d+"',
                 'width="{:.1f}"'.format(width),
                 svg,
                 count=1)
    
    svg = re.sub(r'height="\d+\.\d+"',
                 'heigth="{:.1f}"'.format(height),
                 svg,
                 count=1)
    
    svg = re.sub(r'stroke-width="\d+\.\d+"',
                 'stroke-width="{:f}"'.format(stroke_width),
                 svg)
    
    return svg

rework_svg(secbase._repr_svg_(), 500, 100)

'<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="500.0" heigth="100.0" viewBox="-0.0478752 -0.0974352 1.2958703999999999 0.2842104" preserveAspectRatio="xMinYMin meet"><g transform="matrix(1,0,0,-1,0,0.08933999999999997)"><g><polyline fill="none" stroke="#66cc99" stroke-width="0.003816" points="1.2,-0.0026520000000000003 1.1894879999999999,0.00021600000000000002 1.174308,0.003912 1.1541599999999999,0.008532 1.131624,0.013788 1.108452,0.019343999999999997 1.08516,0.025104 1.0618319999999999,0.031007999999999997 1.0384799999999998,0.037068 1.015104,0.043272 0.9916919999999999,0.049608 0.968232,0.056063999999999996 0.9447,0.06261599999999999 0.921096,0.069228 0.897564,0.075816 0.87426,0.082272 0.8511839999999999,0.088488 0.8283,0.094428 0.8055359999999999,0.100032 0.7828320000000001,0.105276 0.7601399999999999,0.110148 0.737424,0.114636 0.714672,0.118728 0.691836,0.122448 0.6689519999999999,0.12576 0.6459719999999999,0.12869999999999998 0.622944,0

#### massana = structure.MassAnalysis(spar)
massana.massproperties